In [54]:
import requests
import pandas as pd


# 테스트할 FastAPI 서버의 URL
url = "http://127.0.0.1:8000/uploadfile/"

# 테스트용 CSV 파일 경로
test_file = "./data/test.csv"

# 테스트용 .csv 파일 생성
def create_test_csv(file_path: str):
    data = "column1,column2,column3\n1,2,3\n4,5,6\n7,8,9\n"
    with open(file_path, "w") as f:
        f.write(data)
# 테스트 파일 경로
# test_file = "./data/test_file.csv"
# create_test_csv(test_file)


# 파일을 업로드하는 POST 요청
with open(test_file, "rb") as f:
    files = {"file": ("test_file.csv", f, "text/csv")}
    print(files)
    response = requests.post(url, files=files)

    df = pd.read_csv(test_file)
    df.head(3)

if response.status_code == 200:
    print("파일 업로드 성공!")
    print("응답 JSON:", response.json())
else:
    print(f"파일 업로드 실패: {response.status_code}")
    print("응답 내용:", response.text)

{'file': ('test_file.csv', <_io.BufferedReader name='./data/test.csv'>, 'text/csv')}
파일 업로드 성공!
응답 JSON: {'filename': 'test_file.csv', 'dataframe_preview': {'Pclass': {'0': 3, '1': 3, '2': 2, '3': 3, '4': 3}, 'Sex': {'0': 1, '1': 0, '2': 1, '3': 1, '4': 0}, 'Age_x': {'0': 34.5, '1': 47.0, '2': 62.0, '3': 27.0, '4': 22.0}, 'SibSp': {'0': 0, '1': 1, '2': 0, '3': 0, '4': 1}, 'Parch': {'0': 0, '1': 0, '2': 0, '3': 0, '4': 1}, 'Fare': {'0': 7.8292, '1': 7.0, '2': 9.6875, '3': 8.6625, '4': 12.2875}, 'Embarked': {'0': 1, '1': 2, '2': 1, '3': 2, '4': 2}, 'honorific': {'0': 4, '1': 5, '2': 4, '3': 4, '4': 5}, 'alone': {'0': 1, '1': 0, '2': 1, '3': 1, '4': 0}, 'family_num': {'0': 0, '1': 1, '2': 0, '3': 0, '4': 2}}}


In [6]:
from fastapi.testclient import TestClient
from main import app  # FastAPI 애플리케이션 import (main.py에 app이 정의되어 있다고 가정)
import os
import pytest

# 테스트 클라이언트 생성
client = TestClient(app)

# 테스트용 .csv 파일 생성
def create_test_csv(file_path: str):
    data = "column1,column2,column3\n1,2,3\n4,5,6\n7,8,9\n"
    with open(file_path, "w") as f:
        f.write(data)

# 테스트 파일 경로
test_file = "test_file.csv"

# 테스트 실행 전에 파일 생성
create_test_csv(test_file)

# 테스트: 파일 업로드
def test_upload_file():
    with open(test_file, "rb") as f:
        response = client.post("/upload/", files={"file": ("test_file.csv", f, "text/csv")})
    
    # 응답이 정상적으로 반환되었는지 확인
    assert response.status_code == 200

    # 응답 내용이 예상대로 포함되어 있는지 확인
    response_json = response.json()
    assert "filename" in response_json
    assert response_json["filename"] == "test_file.csv"
    assert "dataframe_preview" in response_json
    assert isinstance(response_json["dataframe_preview"], dict)

# 테스트 후 파일 삭제
# os.remove(test_file)


In [1]:
!pip install kagglehub

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mnassrib/telecom-churn-datasets")

print("Path to dataset files:", path)

100%|██████████| 99.5k/99.5k [00:00<00:00, 512kB/s]

Extracting files...
Path to dataset files: /home/dongik/.cache/kagglehub/datasets/mnassrib/telecom-churn-datasets/versions/1


In [6]:
import pandas as pd

df = pd.read_csv(path + '/churn-bigml-80.csv')
df

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,SC,79,415,No,No,0,134.7,98,22.90,189.7,68,16.12,221.4,128,9.96,11.8,5,3.19,2,False
2662,AZ,192,415,No,Yes,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
2663,WV,68,415,No,No,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
2664,RI,28,510,No,No,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2666 entries, 0 to 2665
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   State                   2666 non-null   object 
 1   Account length          2666 non-null   int64  
 2   Area code               2666 non-null   int64  
 3   International plan      2666 non-null   object 
 4   Voice mail plan         2666 non-null   object 
 5   Number vmail messages   2666 non-null   int64  
 6   Total day minutes       2666 non-null   float64
 7   Total day calls         2666 non-null   int64  
 8   Total day charge        2666 non-null   float64
 9   Total eve minutes       2666 non-null   float64
 10  Total eve calls         2666 non-null   int64  
 11  Total eve charge        2666 non-null   float64
 12  Total night minutes     2666 non-null   float64
 13  Total night calls       2666 non-null   int64  
 14  Total night charge      2666 non-null   

In [9]:
df

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,SC,79,415,No,No,0,134.7,98,22.90,189.7,68,16.12,221.4,128,9.96,11.8,5,3.19,2,False
2662,AZ,192,415,No,Yes,36,156.2,77,26.55,215.5,126,18.32,279.1,83,12.56,9.9,6,2.67,2,False
2663,WV,68,415,No,No,0,231.1,57,39.29,153.4,55,13.04,191.3,123,8.61,9.6,4,2.59,3,False
2664,RI,28,510,No,No,0,180.8,109,30.74,288.8,58,24.55,191.9,91,8.64,14.1,6,3.81,2,False
